# 1 

In [52]:
# код для поиска w_map
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
class Posterior(nn.Module):
    def __init__(self, A, vec_size = 10):
        super().__init__()
        self.vec_size = vec_size

        w_init = np.random.multivariate_normal(mean = np.zeros((vec_size)), cov= A)
        self.w = torch.tensor(w_init, requires_grad=True, dtype=torch.float32)
        # self.w.requires_grad = True

    def forward(self, X, A, y):
        sigm_elems = y * (X @ self.w)
        sigmas = F.logsigmoid(sigm_elems)
        rez = torch.sum(sigmas) - 0.5 * self.w.T @ A @ self.w
        return rez
    def predict_proba(self, X):
        sigm_elems = (X @ self.w)
        sigmas = F.sigmoid(sigm_elems)
        return sigmas

def sample_predict(X_test, X_train, y_train, A):
    # аппроксимируем правильный подсчет вероятностей
    poster_coeff = np.exp(-0.5 * np.sum(X_train * X_train))
    def compute_posterior(X_train, y_train, w):
        sigm_elems = y * (X @ self.w)
        sigmas = F.logsigmoid(sigm_elems)
        rez = torch.sum(sigmas) - 0.5 * self.w.T @ A @ self.w
        rez pos
        return np.prod()
    def one_sample():
        w_sampler = multivariate_normal(mean = np.zeros(vec_size), cov = A)
        tmp = w_sampler.rvs(1000)
        probs = w_sampler.pdf(tmp)
        sigmas = 1 / (1 + np.exp(-X))
    w_s = np.random.multivariate_normal(mean = np.zeros((vec_size)), cov= A, size= (1000))


In [123]:
from scipy.stats import multivariate_normal

A = np.eye(vec_size)
norm = multivariate_normal(mean = np.zeros(vec_size), cov = A)
tmp = norm.rvs(3)
print(tmp)
print(norm.pdf(np.zeros(vec_size)))

[[ 3.26971333e-01  1.28395162e+00 -1.28424775e+00 -1.98607884e-01
   1.89301360e+00  4.08555776e-01 -6.55319680e-01 -4.76188721e-01
  -1.47161947e+00  8.68741127e-01]
 [ 6.29300769e-01 -5.62647880e-01 -3.91498665e-01  1.05022357e-01
   7.60061631e-01  5.93317383e-02 -2.93036352e-01  6.67507234e-01
   1.05934136e+00  5.01132476e-01]
 [ 1.69259052e+00  9.45203034e-01 -1.01075418e+00  1.58645900e+00
  -7.58025351e-01  1.33516372e+00  3.85963320e-01 -3.70509957e-01
  -5.12454628e-01 -1.66532177e-03]]
0.00010211761384541846


In [98]:
import numpy as np

vec_size = 10
elems = 100
A = np.eye(vec_size)
A = torch.Tensor(A)
X = torch.rand((100, vec_size))
y = torch.rand((elems)) > 0.3
poster = posterior(A, vec_size)

optim = torch.optim.AdamW([poster.w],)

In [99]:
for i in range(100000):
    tmp = -poster(X, A, y)
    optim.zero_grad()
    tmp.backward()
    optim.step()
    if i % 10000 == 0:
        print(tmp.detach().item())

167.8400115966797
27.5008487701416
27.5008487701416
27.50084686279297
27.50084686279297
27.50084686279297
27.50084686279297
27.5008487701416
27.50084686279297
27.50084686279297


In [100]:
poster.w

tensor([0.7804, 0.7265, 0.6909, 0.8099, 0.6750, 0.8989, 0.8006, 0.7572, 0.6551,
        0.7853], requires_grad=True)

In [102]:
torch.mean((poster.predict_proba(X) > 0.95) * 1.)

tensor(0.8900)